In [1]:
import pandas as pd
import os
import geopandas as gpd
import glob

os.chdir("/home/ubuntu/real_estate_dashboard")
print(os.getcwd())

/home/ubuntu/real_estate_dashboard


# 법정동 코드 지리 데이터 정리

In [12]:
legal_info_b = pd.read_csv("0_data/legal_info_b.csv")
legal_info_b = legal_info_b.astype({'법정동코드':'str',
  '법정동시군구코드':'str',
  '법정동읍면동코드':'str'})

legal_info_b['법정동코드_2'] = legal_info_b['법정동코드'].str[:8] + '00'

legal_info_b.head()

In [11]:
geo_data = gpd.read_file('0_data/geoservice/emd.shp', dtype={'EMD_CD':object}, encoding='utf-8')
geo_data['EMD_CD'] = geo_data['EMD_CD'] + "00"
geo_data['EMD_KOR_NM'] = geo_data['EMD_KOR_NM'].str.decode('euc-kr')

geo_data.rename(columns = {'EMD_CD':'법정동코드_2'}, inplace=True)
geo_data = geo_data.astype({'법정동코드_2':'str'})
geo_data = geo_data[['법정동코드_2', 'geometry']]

geo_data.head()

In [13]:
legal_geo_info = pd.merge(legal_info_b, geo_data, 
                          on = "법정동코드_2",
                          how = 'left')

legal_geo_info.head(3).T

In [42]:
#  key: 법정동코드
legal_geo_info_2 = legal_geo_info[['법정동코드','시도명', '시군구명','동리명', 'geometry']].drop_duplicates()

# key: 법정동시군구코드
legal_geo_info_3 = legal_geo_info[['법정동시군구코드','시도명', '시군구명']].drop_duplicates()

# key: 주소
legal_geo_info_4 = legal_geo_info[['주소', '법정동코드', '법정동읍면동코드','geometry']].drop_duplicates()

## 법정동 지리데이터 통합

In [3]:
legal_info_b = pd.read_csv("0_data/legal_info_b.csv")
legal_info_b = legal_info_b.astype({'법정동코드':'str',
  '법정동시군구코드':'str',
  '법정동읍면동코드':'str'})

legal_info_b['법정동코드_2'] = legal_info_b['법정동코드'].str[:8] + '00'

geo_data = gpd.read_file('0_data/geoservice/emd.shp', dtype={'EMD_CD':object}, encoding='utf-8')
geo_data['EMD_CD'] = geo_data['EMD_CD'] + "00"
geo_data['EMD_KOR_NM'] = geo_data['EMD_KOR_NM'].str.decode('euc-kr')

geo_data.rename(columns = {'EMD_CD':'법정동코드_2'}, inplace=True)
geo_data = geo_data.astype({'법정동코드_2':'str'})
geo_data = geo_data[['법정동코드_2', 'geometry']]

legal_geo_info = pd.merge(legal_info_b, geo_data, 
                          on = "법정동코드_2",
                          how = 'left')

#  key: 법정동코드
legal_geo_info_2 = legal_geo_info[['법정동코드','시도명', '시군구명','동리명', 'geometry']].drop_duplicates()

# key: 법정동시군구코드
legal_geo_info_3 = legal_geo_info[['법정동시군구코드','시도명', '시군구명']].drop_duplicates()

# key: 주소
legal_geo_info_4 = legal_geo_info[['주소', '법정동코드', '법정동읍면동코드','geometry']].drop_duplicates()

## 데이터 불러오기

In [7]:
# 아파트 매매 실거래
apt_trade = pd.read_csv("0_data/molit/apt_trade/apt_trade_202101.csv", thousands=r',')
# 아파트 전월세 
apt_rent = pd.read_csv("0_data/molit/apt_rent/apt_rent_202101.csv", thousands=r',')

# 연립 다세대 매매 실거래
rh_trade = pd.read_csv("0_data/molit/rh_trade/rh_trade_202101.csv", thousands=r',')
# 연립 다세대 전월세
rh_rent = pd.read_csv("0_data/molit/rh_rent/rh_rent_202101.csv", thousands=r',')

# 단독/다가구 매매 실거래
sh_trade = pd.read_csv("0_data/molit/sh_trade/sh_trade_202101.csv", thousands=r',')
# 단독/다가구 전월세
sh_rent = pd.read_csv("0_data/molit/sh_rent/sh_rent_202101.csv", thousands=r',')

# 오피스텔 매매 실거래
offi_trade = pd.read_csv("0_data/molit/offi_trade/offi_trade_202101.csv", thousands=r',')
# 오피스텔 전월세
offi_rent = pd.read_csv("0_data/molit/offi_rent/offi_rent_202101.csv", thousands=r',')

# 아파트 매매 실거래

In [10]:
apt_trade['법정동코드'] = apt_trade['법정동시군구코드'].astype(str) + apt_trade['법정동읍면동코드'].astype(str)
apt_trade = apt_trade[['거래금액', '거래유형', '건축년도', '전용면적', '법정동','법정동코드','아파트','층','년','월','일']]

In [14]:
apt_trade_final = pd.merge(apt_trade, legal_geo_info_2, 
                        on = "법정동코드",
                        how = 'left')
apt_trade_final.head(3)

In [13]:
apt_trade_final = apt_trade_final.astype({
    '거래금액':'int',
    '거래유형':'object',
    '건축년도':'int',
    '전용면적':'float',
    '법정동':'object',
    '법정동코드':'object',
    '아파트':'object',
    '층':'int',
    '년':'int',
    '월':'int',
    '일':'int',
    '시도명': 'object',
    '시군구명': 'object',
    '동리명': 'object',
    'geometry': 'geometry',
})

apt_trade_final.dtypes

거래금액           int64
거래유형          object
건축년도          object
전용면적         float64
법정동           object
법정동코드         object
아파트           object
층              int64
년              int64
월              int64
일              int64
시도명           object
시군구명          object
동리명           object
geometry    geometry
dtype: object

## 아파트 실거래가 최종

In [53]:
# 법정동시군구코드, 건축년도 자료형 변경
apt_trade = apt_trade[apt_trade['건축년도'].notnull()]
apt_trade = apt_trade.astype({'법정동시군구코드':'str',
                              '법정동읍면동코드':'str',
                              '건축년도':'int'})
    
apt_trade['법정동코드'] = apt_trade['법정동시군구코드'] + apt_trade['법정동읍면동코드']

apt_trade = apt_trade[['거래금액', '거래유형', 
                       '건축년도', '전용면적', 
                       '법정동','법정동코드','아파트','층',
                       '년','월','일']]
    
apt_trade_final = pd.merge(apt_trade, legal_geo_info_2, 
                        on = "법정동코드",
                        how = 'left')

apt_trade_final = apt_trade_final.astype({
    '거래금액':'int',
    '거래유형':'object',
    '건축년도':'int',
    '전용면적':'float',
    '법정동':'object',
    '법정동코드':'object',
    '아파트':'object',
    '층':'int',
    '년':'int',
    '월':'int',
    '일':'int',
    '시도명': 'object',
    '시군구명': 'object',
    '동리명': 'object',
    'geometry': 'geometry',
})

In [1]:
# legal_geo_info_2

# 아파트 전월세 예시

In [21]:
apt_rent = apt_rent[['건축년도', '법정동', '지번','아파트','보증금액','월세금액','전용면적','층','지역코드','년','월','일']]

apt_rent.rename(columns = {'지역코드':'법정동시군구코드'}, inplace=True)
apt_rent.rename(columns = {'법정동':'동리명'}, inplace=True)
apt_rent = apt_rent.astype({'법정동시군구코드':'str'})

apt_rent_2 = pd.merge(apt_rent, legal_geo_info_3, 
                      on = ['법정동시군구코드'],
                      how = 'left')

In [23]:
apt_rent_2['시도명'] = apt_rent_2['시도명'].str.strip()
apt_rent_2['시군구명'] = apt_rent_2['시군구명'].str.strip()
apt_rent_2['동리명'] = apt_rent_2['동리명'].str.strip()

apt_rent_2['주소'] = apt_rent_2['시도명'] + " " + apt_rent_2['시군구명'] + " " + apt_rent_2['동리명']

In [162]:
apt_rent_3 = pd.merge(apt_rent_2, legal_geo_info_4, 
                      on = ['주소'],
                      how = 'left')

In [166]:
apt_rent_3 = apt_rent_3.replace('충청북도 청주시 상당구 북문로2가동', '충청북도 청주시 상당구 북문로2가')
apt_rent_3 = apt_rent_3.replace('충청북도 청주시 상당구 북문로3가동', '충청북도 청주시 상당구 북문로3가')
apt_rent_3 = apt_rent_3.replace('충청북도 청주시 상당구 남문로1가동', '충청북도 청주시 상당구 남문로1가')

# 아파트 전월세

In [63]:
apt_rent = pd.read_csv("0_data/molit/apt_rent/apt_rent_202101.csv", thousands=r',')

apt_rent.rename(columns = {'지역코드':'법정동시군구코드'}, inplace=True)
apt_rent.rename(columns = {'법정동':'동리명'}, inplace=True)

# 법정동시군구코드, 건축년도 자료형 변경
apt_rent = apt_rent[apt_rent['건축년도'].notnull()]
apt_rent = apt_rent.astype({'법정동시군구코드':'str',
               '건축년도':'int'})

apt_rent_2 = pd.merge(apt_rent, legal_geo_info_3, 
                      on = ['법정동시군구코드'],
                      how = 'left')

apt_rent_2['시도명'] = apt_rent_2['시도명'].str.strip()
apt_rent_2['시군구명'] = apt_rent_2['시군구명'].str.strip()
apt_rent_2['동리명'] = apt_rent_2['동리명'].str.strip()

apt_rent_2 = apt_rent_2.where(pd.notnull(apt_rent_2), '')

apt_rent_2['주소'] = apt_rent_2['시도명'] + " " + \
                     apt_rent_2['시군구명'] + " " + \
                     apt_rent_2['동리명']

In [71]:
apt_rent = pd.read_csv("0_data/molit/apt_rent/apt_rent_202111.csv", thousands=r',')

apt_rent.rename(columns = {'지역코드':'법정동시군구코드'}, inplace=True)
apt_rent.rename(columns = {'법정동':'동리명'}, inplace=True)

# 법정동시군구코드, 건축년도 자료형 변경
apt_rent = apt_rent[apt_rent['건축년도'].notnull()]
apt_rent = apt_rent.astype({'법정동시군구코드':'str',
               '건축년도':'int'})

apt_rent_2 = pd.merge(apt_rent, legal_geo_info_3, 
                      on = ['법정동시군구코드'],
                      how = 'left')

apt_rent_2['시도명'] = apt_rent_2['시도명'].str.strip()
apt_rent_2['시군구명'] = apt_rent_2['시군구명'].str.strip()
apt_rent_2['동리명'] = apt_rent_2['동리명'].str.strip()

apt_rent_2 = apt_rent_2.where(pd.notnull(apt_rent_2), '')

apt_rent_2['주소'] = apt_rent_2['시도명'] + " " + apt_rent_2['시군구명'] + " " + apt_rent_2['동리명']

apt_rent_2['주소'] = apt_rent_2['주소'].str.replace('  ', ' ')
apt_rent_2['주소'] = apt_rent_2['주소'].str.strip()

apt_rent_2 = apt_rent_2.replace('충청북도 청주시 상당구 북문로2가동', '충청북도 청주시 상당구 북문로2가')
apt_rent_2 = apt_rent_2.replace('충청북도 청주시 상당구 북문로3가동', '충청북도 청주시 상당구 북문로3가')
apt_rent_2 = apt_rent_2.replace('충청북도 청주시 상당구 남문로1가동', '충청북도 청주시 상당구 남문로1가')

apt_rent_3 = pd.merge(apt_rent_2, legal_geo_info_4, 
                      on = ['주소'],
                      how = 'left')

In [15]:
# apt_rent[apt_rent['법정동시군구코드'].str.contains('43130')]
apt_rent = pd.read_csv("0_data/molit/apt_rent/apt_rent_202111.csv", thousands=r',')


## 함수 만들기

In [5]:
def molit_data_cleaning(df):

    # 아파트 실거래 매매 데이터 전처리
    if('법정동시군구코드' in df.columns):
        # 법정동시군구코드, 건축년도 자료형 변경
        df = df[df['건축년도'].notnull()]
        df = df.astype({'법정동시군구코드':'str',
                        '법정동읍면동코드':'str',
                        '건축년도':'int'})

        df['법정동코드'] = df['법정동시군구코드'] + df['법정동읍면동코드']
        df = df[['거래금액', '거래유형', 
                 '건축년도', '전용면적', 
                 '법정동','법정동코드','아파트','층',
                 '년','월','일']]

        df_final = pd.merge(df, legal_geo_info_2, 
                                on = "법정동코드",
                                how = 'left')
        
    # 아파트 실거래 매매 제외 전부
    else:

        # 컬럼 이름 변경
        df.rename(columns = {'지역코드':'법정동시군구코드'}, inplace=True)
        df.rename(columns = {'법정동':'동리명'}, inplace=True)

        # 법정동시군구코드, 건축년도 자료형 변경
        df = df[df['건축년도'].notnull()]
        df = df.astype({'법정동시군구코드':'str',
                       '건축년도':'int'})

        # legal_geo_info_3과 merge 
        # key = 법정동시군구코드
        df = pd.merge(df, legal_geo_info_3, 
                             on = ['법정동시군구코드'],
                             how = 'left')

        # 시도명, 시군구명, 동리명 양쪽 공백 제거
        df['시도명'] = df['시도명'].str.strip()
        df['시군구명'] = df['시군구명'].str.strip()
        df['동리명'] = df['동리명'].str.strip()

        # NA 길이 없는 문자열로 변경
        df = df.where(pd.notnull(df), '')

        # 시도명, 시군구명, 동리명을 합쳐서 주소 컬럼 추가
        df['주소'] = df['시도명'] + " " + \
                     df['시군구명'] + " " + \
                     df['동리명']

        # 주소컬럼에 공백2개 있는부분 1개로 변경
        # 주소의 양쪽 공백 제거
        df['주소'] = df['주소'].str.replace('  ', ' ')
        df['주소'] = df['주소'].str.strip()

        # 특별한 데이터 문자열 변경
        df = df.replace('충청북도 청주시 상당구 북문로2가동', 
                        '충청북도 청주시 상당구 북문로2가')
        df = df.replace('충청북도 청주시 상당구 북문로3가동', 
                        '충청북도 청주시 상당구 북문로3가')
        df = df.replace('충청북도 청주시 상당구 남문로1가동', 
                        '충청북도 청주시 상당구 남문로1가')

        # legal_geo_info_4와 최종 merge. key 값은 주소  
        df_final = pd.merge(df, legal_geo_info_4, 
                             on = ['주소'],
                             how = 'left')
    
    return(df_final)

## 함수 테스트

In [16]:
df_clean = molit_data_cleaning(apt_trade)

df_clean.head(3)

In [17]:
df_clean = molit_data_cleaning(apt_rent)

df_clean.head(3)

In [18]:
df_clean = molit_data_cleaning(rh_trade)

df_clean.head(3)

In [19]:
df_clean = molit_data_cleaning(rh_rent)

df_clean.head(3)

In [20]:
df_clean = molit_data_cleaning(sh_trade)

df_clean.head(3)

In [21]:
df_clean = molit_data_cleaning(sh_rent)

df_clean.head(3)

In [23]:
df_clean = molit_data_cleaning(offi_trade)

df_clean.head(3)

In [24]:
df_clean = molit_data_cleaning(offi_rent)

df_clean.head(3)

# 저장

In [93]:
# 0_data/molit/apt_trade/apt_trade_202101.csv
apt_trade_test = molit_data_cleaning(apt_trade)
apt_trade_test.shape

(62614, 15)

In [91]:
file_name = f'0_data/cleaning/molit/apt_trade_test.csv'
apt_trade_test.to_csv(file_name, index = False)

In [94]:
# 0_data/molit/apt_trade/apt_trade_202101.csv
apt_trade_test_rm_geometry = molit_data_cleaning(apt_trade)
apt_trade_test_rm_geometry = apt_trade_test_rm_geometry.drop(columns=['geometry'])

file_name = f'0_data/cleaning/molit/apt_trade_test_rm_geometry.csv'
apt_trade_test_rm_geometry.to_csv(file_name, index = False)

In [25]:
# 국토교통부 8개 데이터셋
list_nm = ['apt_trade','apt_rent', 
           'rh_trade', 'rh_rent', 
           'sh_trade', 'sh_rent', 
           'offi_trade', 'offi_rent']


# 8개 데이터셋 반복문
for k in range(len(list_nm)):
    
    dir_list = f'0_data/molit/{list_nm[k]}/*.csv'
    print(dir_list)

In [9]:
# 국토교통부 8개 데이터셋
list_nm = ['apt_trade','apt_rent', 
           'rh_trade', 'rh_rent', 
           'sh_trade', 'sh_rent', 
           'offi_trade', 'offi_rent']

list_nm = ['apt_rent']

# 8개 데이터셋 반복문
for k in range(len(list_nm)):
    
    total = pd.DataFrame()
    
    
    # 8개 데이터셋에 따른 파일 리스트 조회
    dir_list = f'0_data/molit/{list_nm[k]}/*.csv'
    file_list = sorted(glob.glob(dir_list))
    print(file_list)
    

In [10]:
dir_list = f'0_data/{list_nm[k]}/*.csv'
file_list = sorted(glob.glob(dir_list))
print(file_list)
    


[]


In [26]:
file_nm = '0_data/molit/apt_rent/apt_rent_202111.csv'

df = pd.read_csv(file_nm, thousands=r',')

clean_df = molit_data_cleaning(df)
clean_df = clean_df.drop(columns=['geometry'])

total = pd.concat([total, clean_df])

In [43]:
file_nm = '0_data/molit/apt_rent/apt_rent_202111.csv'

df = pd.read_csv(file_nm, thousands=r',')

In [46]:
df['지역코드'] = df['지역코드'].astype(str)

In [13]:
apt_rent_clean = pd.read_csv("0_data/cleaning/molit/apt_rent.csv", thousands=r',')

/tmp/ipykernel_2558/3916799158.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  apt_rent_clean = pd.read_csv("0_data/cleaning/molit/apt_rent.csv", thousands=r',')


In [6]:
# 국토교통부 8개 데이터셋
list_nm = ['apt_trade','apt_rent', 
           'rh_trade', 'rh_rent', 
           'sh_trade', 'sh_rent', 
           'offi_trade', 'offi_rent']

list_nm = ['offi_rent']

# 8개 데이터셋 반복문
for k in range(len(list_nm)):
    
    total = pd.DataFrame()
    
    
    # 8개 데이터셋에 따른 파일 리스트 조회
    dir_list = f'0_data/{list_nm[k]}/*.csv'
    file_list = sorted(glob.glob(dir_list))
    
    # 각 파일 리스트에 대한 반복문
    # molit_data_cleaning() 함수를 사용한 전처리
    # geometry 컬럼 제거
    # total 에다가 모두 이어붙임
    for i in range(len(file_list)):
        print(i)
        df = pd.read_csv(file_list[i], thousands=r',')
        
        clean_df = molit_data_cleaning(df)
        clean_df = clean_df.drop(columns=['geometry'])

        total = pd.concat([total, clean_df])

    # 최종적으로 만든 total은 0_data_claeaning/molit에 저장
    file_name = f'0_data/cleaning/molit/{list_nm[k]}_2.csv'
    total.to_csv(file_name, index = False)

In [62]:
# 국토교통부 8개 데이터셋
list_nm = ['apt_trade','apt_rent', 
           'rh_trade', 'rh_rent', 
           'sh_trade', 'sh_rent', 
           'offi_trade', 'offi_rent']

# 8개 데이터셋 반복문
for k in range(len(list_nm)):
    
    total = pd.DataFrame()
    
    
    # 8개 데이터셋에 따른 파일 리스트 조회
    dir_list = f'0_data/molit/{list_nm[k]}/*.csv'
    file_list = sorted(glob.glob(dir_list))
    
    # 각 파일 리스트에 대한 반복문
    # molit_data_cleaning() 함수를 사용한 전처리
    # geometry 컬럼 제거
    # total 에다가 모두 이어붙임
    for i in range(len(file_list)):
        print(i)
        df = pd.read_csv(file_list[i], thousands=r',')
        
        clean_df = molit_data_cleaning(df)
        clean_df = clean_df.drop(columns=['geometry'])

        total = pd.concat([total, clean_df])

    # 최종적으로 만든 total은 0_data_claeaning/molit에 저장
    file_name = f'0_data/cleaning/molit/{list_nm[k]}.csv'
    total.to_csv(file_name, index = False)

In [74]:
df1 = pd.read_csv('0_data/cleaning/molit/apt_trade.csv', dtype = {'법정동코드':str})

/home/ubuntu/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [77]:
df1

,거래금액,거래유형,건축년도,전용면적,법정동,법정동코드,아파트,층,년,월,일,시도명,시군구명,동리명
0,105000,,2008,97.6100,사직동,1111011500,광화문스페이스본(101동~105동),10,2019,1,7,서울특별시,종로구,NaN
1,162000,,2008,163.3300,사직동,1111011500,광화문스페이스본(106동),11,2019,1,11,서울특별시,종로구,NaN
2,119000,,2008,131.4400,사직동,1111011500,광화문스페이스본(101동~105동),13,2019,1,19,서울특별시,종로구,NaN
3,119000,,2008,147.3100,사직동,1111011500,광화문스페이스본(101동~105동),1,2019,1,30,서울특별시,종로구,NaN
4,160000,,2004,150.4800,내수동,1111011800,경희궁의아침4단지,14,2019,1,25,서울특별시,종로구,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241160,7950,직거래,2000,58.9440,대정읍 상모리,5013025021,한빛테크,2,2022,11,4,제주특별자치도,서귀포시,상모리
2241161,28500,직거래,2016,80.0500,대정읍 상모리,5013025021,해누리2차아파트,2,2022,11,4,제주특별자치도,서귀포시,상모리
2241162,10500,중개거래,1996,60.0000,남원읍 남원리,5013025321,에덴,5,2022,11,17,제주특별자치도,서귀포시,남원리
2241163,31500,중개거래,2017,80.9348,표선면 표선리,5013032021,다움채,4,2022,11,1,제주특별자치도,서귀포시,표선리


In [81]:
legal_geo_info = pd.merge(legal_info_b, geo_data, 
                          on = "법정동코드_2",
                          how = 'left')

df2 = pd.merge(df1, legal_geo_info, 
               on = "법정동코드",
               how = 'left')
df2 = pd.merge(df1, legal_geo_info_2[['법정동코드', 'geometry']], 
                        on = "법정동코드",
                        how = 'left')


# S3

In [1]:
# file_manager.py
import os
import boto3

# apart-admin
AWS_ACCESS_KEY_ID = "IAM Access Key"
AWS_ACCESS_KEY_SECRET = "IAM Secret Key"

# set aws credentials 
s3r = boto3.resource('s3', aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_ACCESS_KEY_SECRET)
bucket = s3r.Bucket('apart-bucket')

In [4]:
# downloading folder 
prefix = '0_data'
for object in bucket.objects.filter(Prefix = '0_data'):
    print(object.key)

In [106]:
list_nm = ['apt_trade','apt_rent', 'rh_trade', 'rh_rent', 'sh_trade', 'sh_rent', 'offi_trade', 'offi_rent']

In [5]:
for list_value in list_nm:
#     list_value = 'apt_rent'
    print(f'0_data/cleaning/{list_value}_201901.csv')
    print(f'0_data_raw/{list_value}/{list_value}_201901.csv')

In [43]:
for list_value in list_nm:
    bucket.download_file(f'0_data_raw/{list_value}/{list_value}_201901.csv',f'0_data/cleaning/{list_value}_201901.csv')